    Ben Christensen
    Math 321
    10/26/17

Create a tree object and implement the breadth first search algorithm. Use it to solve the degrees to Kevin Bacon Problem for various actors, and the average degrees to Kevin Bacon. This can be used to even find the degrees of separation from any actor in the dataset, not just Kevin Bacon

In [2]:
import pdb
from collections import deque
import networkx as nx
from matplotlib import pyplot as plt



In [3]:
# Problems 1-3: Implement the following class
class Graph(object):
    """A graph object, stored as an adjacency dictionary. Each node in the
    graph is a key in the dictionary. The value of each key is a list of the
    corresponding node's neighbors.

    Attributes:
        dictionary: the adjacency list of the graph.
    """

    def __init__(self, adjacency):
        """Store the adjacency dictionary as a class attribute."""
        self.dictionary = adjacency

    # Problem 1
    def __str__(self):
        """String representation: a view of the adjacency dictionary.

        Example:
            >>> test = {'A':['B'], 'B':['A', 'C',], 'C':['B']}
            >>> print(Graph(test))
            A: B
            B: A; C
            C: B
        """
        string = ""
        for each in self.dictionary.keys():
            separator = "; "
            string += each + ": " + separator.join(self.dictionary[each]) + "\n"
        string = string.strip()
        return string


    # Problem 2
    def traverse(self, start):
        """Begin at 'start' and perform a breadth-first search until all
        nodes in the graph have been visited. Return a list of values,
        in the order that they were visited.

        Parameters:
            start: the node to start the search at.

        Returns:
            the list of visited nodes (in order of visitation).

        Raises:
            ValueError: if 'start' is not in the adjacency dictionary.

        Example:
            >>> test = {'A':['B'], 'B':['A', 'C',], 'C':['B']}
            >>> Graph(test).traverse('B')
            ['B', 'A', 'C']
        """
        if start not in self.dictionary.keys():
            raise ValueError("Starting node not in the graph")
        current = start
        visit_queue = deque()
        visited = list()
        marked = set()
        visited.append(current)
        marked.add(current)
        #pdb.set_trace()
        while len(visited) != len(self.dictionary.keys()):
            for neighbor in self.dictionary[current]:
                if neighbor not in marked:
                    visit_queue.append(neighbor)
                    marked.add(neighbor)
            current = visit_queue.popleft()
            visited.append(current)
        return visited




    # Problem 3
    def shortest_path(self, start, target):
        """Begin at the node containing 'start' and perform a breadth-first
        search until the node containing 'target' is found. Return a list
        containg the shortest path from 'start' to 'target'. If either of
        the inputs are not in the adjacency graph, raise a ValueError.

        Parameters:
            start: the node to start the search at.
            target: the node to search for.

        Returns:
            A list of nodes along the shortest path from start to target,
                including the endpoints.

        Example:
            >>> test = {'A':['B', 'F'], 'B':['A', 'C'], 'C':['B', 'D'],
            ...         'D':['C', 'E'], 'E':['D', 'F'], 'F':['A', 'E', 'G'],
            ...         'G':['A', 'F']}
            >>> Graph(test).shortest_path('A', 'G')
            ['A', 'F', 'G']
        """
        if start not in self.dictionary.keys() or target not in self.dictionary.keys():
            raise ValueError("Both start and target nodes must exist in the graph")
        #pdb.set_trace()
        current = start
        visit_queue = deque()
        visited = list()
        marked = set()
        tracker = dict()
        visited.append(current)
        marked.add(current)
        while len(visited) != len(self.dictionary.keys()):
            for neighbor in self.dictionary[current]:
                if neighbor not in marked:
                    visit_queue.append(neighbor)
                    marked.add(neighbor)
                    tracker[neighbor] = current
            current = visit_queue.popleft()
            visited.append(current)
        path = [target]
        current = target
        if target == start:
            return [start]
        while current != start:
            path += tracker[current]
            current = tracker[current]
        return path[::-1]


# Problem 4: Write the following function
def convert_to_networkx(dictionary):
    """Convert 'dictionary' to a networkX object and return it."""
    graph = nx.Graph()
    for key in dictionary.keys():
        for neighbor in dictionary[key]:
            graph.add_edge(key, neighbor)
    return graph



# Problems 5-7: Implement the following class
class BaconSolver(object):
    """Class for solving the Kevin Bacon problem."""

    # Problem 5
    def __init__(self, filename="/Users/benchristensen/Desktop/ACME Python Labs/Volume2-Student-Materials/BreadthFirstSearch/movieData.txt"):
        """Initialize the networkX graph with data from the specified
        file. Store the graph as a class attribute. Also store the collection
        of actors in the file as an attribute.
        """
        mDict = dict()
        actors = set()
        with open(filename, 'r') as myFile:
            lines = myFile.readlines()
        for line in lines:
            line = line.strip().split('/')
            mDict[line[0]] = line[1:]
            for actor in mDict[line[0]]:
                actors.add(actor)
        self.actors = actors
        self.netX = convert_to_networkx(mDict)

    # Problem 6
    def path_to_bacon(self, start, target="Bacon, Kevin"):
        """Find the shortest path from 'start' to 'target'."""
        if start not in self.actors or target not in self.actors:
            raise ValueError("Both start and target must be in the list of actors")
        return nx.shortest_path(self.netX, start, target)

    # Problem 7
    def bacon_number(self, start, target="Bacon, Kevin"):
        """Return the Bacon number of 'start'."""
        path = self.path_to_bacon(start, target)
        for each in path:
            if each not in self.actors:
                path.remove(each)
        return len(path) - 1

    # Problem 7
    def average_bacon(self, target="Bacon, Kevin"):
        """Calculate the average Bacon number in the data set.
        Note that actors are not guaranteed to be connected to the target.

        Parameters:
            target (str): the node to search the graph for.

        Returns:
            (float): the average (finite) Bacon number
            (int): the number of actors not connected to the target.
        """
        baconNums = []
        failures = 0
        for actor in self.actors:
            if nx.has_path(self.netX, actor, target):
                baconNums.append(self.bacon_number(actor, target))
            else:
                failures += 1
        return sum(baconNums) / len(baconNums), failures




In [4]:
test = {'A':['B', 'F'], 'B':['A', 'C'], 'C':['B', 'D'],
        'D':['C', 'E'], 'E':['D', 'F'], 'F':['A', 'E', 'G'],
        'G':['A', 'F']}
print(Graph(test).shortest_path('B', 'F'))

['B', 'A', 'F']


In [11]:
Solver = BaconSolver()

In [12]:
sol = Solver.average_bacon()
print("Average degrees to Kevin Bacon: ", sol[0])
print("Number of actors not connected to Kevin Bacon: ", sol[1])
print("Rafael Zubizarreta's degrees from Kevin Bacon: ", Solver.bacon_number("Zubizarreta, Rafael"))
print("How Rafael Zubizarreta is connected to Kevin Bacon: ", Solver.path_to_bacon("Zubizarreta, Rafael"))

Average degrees to Kevin Bacon:  2.666478488932797
Number of actors not connected to Kevin Bacon:  847
Rafael Zubizarreta's degrees from Kevin Bacon:  2
How Rafael Zubizarreta is connected to Kevin Bacon:  ['Zubizarreta, Rafael', 'The Dark Knight Rises', 'King, Joey', 'Crazy, Stupid, Love.', 'Bacon, Kevin']
